In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from scipy.interpolate import make_interp_spline, BSpline
from cycler import cycler
from pathlib import Path
%matplotlib notebook

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df.head()

,ball,probability,outcome,runs,wickets,run_rate,to_win
0,1,58.15,0,0,0,0.0,NaN
1,2,56.99,0,0,0,0.0,NaN
2,3,58.86,1,1,0,2.0,NaN
3,4,61.99,1,2,0,3.0,NaN
4,5,56.92,1,3,0,3.6,NaN


In [4]:
ball = df.ball
prob = df.probability
outcomes = df.outcome
runs = df.runs
wickets = df.wickets
run_rate = df.run_rate
to_win = df.to_win

In [5]:
## Initialization

# Define x and y
x = ball
y = prob

## If you want your curve to be smoothened,
# x = np.linspace(ball.min(), ball.max(), 200)
# spl = make_interp_spline(ball, prob, k=3)  # type: BSpline
# power_smooth = spl(x)
# y = power_smooth

In [6]:
# Actual Curve (It's better if you have data for each ball)

# Resets the rcParams before running each time
plt.rcParams.update(plt.rcParamsDefault)

plt.rcParams.update({
    "axes.prop_cycle": cycler(color=["#FEE715FF", "#e94cdc", "0.7"]),
    "lines.linewidth": 3,
    "axes.titlecolor": "white",
    "axes.facecolor": "#101820FF",
    "axes.edgecolor": "white",
    "axes.labelcolor": "white",
    "xtick.color": "white",
    "ytick.color": "white",
    "figure.facecolor": "#101820FF",
    "savefig.facecolor": "#101820FF"
})

# Creating the figure
fig, ax = plt.subplots()
line, = ax.plot(x, y)

## Plot Customization

# Use a suitable font. If 'Oswald' font family is not available, you can download it by visiting https://fonts.google.com/specimen/Oswald
font = {'fontname':'Oswald'}
axis_font = {'fontname':'Oswald Regular'}
subtitle_font = {'fontname':'Tw Cen MT Condensed'}

# Label/Title definitions and styling
fig.subplots_adjust(top=0.85)
fig.suptitle("Sri Lanka's Win Probability Variation vs Australia ( T20I - SL vs AUS on $\mathregular{11^{th}}$ June 2022 ) \n Classifier: Logistic Regression", fontsize=15, color='white', y=0.96, **font)
plt.xlabel('Ball', fontsize = 13, **axis_font, labelpad = 20)
plt.ylabel('Win Probability (%)', fontsize = 13, **axis_font, labelpad = 20)
fig.set_size_inches(16, 9)
fig.set_tight_layout({"pad": 7})
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xticks(np.arange(6, 246, 6),**axis_font)
plt.yticks(np.arange(10, 110, 10),**axis_font)
ax.set_ylim([0, 100])
line.axes.axis([0, 246, 0, 110])

# Adding a line to indicate the end of the first innings at x = 120, and the 50% probability line
plt.axvline(x=120, color='white', linestyle='--', label = 'End of First Innings', linewidth=1.5)  
plt.axhline(y=50, color='white', linestyle='--', label = 'End of First Innings', linewidth=1.5)  

# Definining the text elements and text boxes which will be dynamically updated
result = plt.figtext(0.25,0.95, "", bbox={'facecolor':'w', 'alpha':0.8, 'pad':1.2, 'boxstyle':'round', 'edgecolor': '#101820FF'}, transform=ax.transAxes, ha="center", color = 'white', **font)
t1 = ax.text(0,1.05, "", transform = ax.transAxes, ha="left", color='yellow', fontsize=17, **subtitle_font)
t2 = ax.text(0.5,1.05, "", transform = ax.transAxes, ha="center", color='yellow', fontsize=17, **subtitle_font)
t3 = ax.text(1,1.05, "", transform = ax.transAxes, ha="right", color='yellow', fontsize=17, **subtitle_font)

# Footer text to add credits/URLs
plt.figtext(0.99, 0.01, 'predictors.shinyapps.io/wpsg/', horizontalalignment='right', color = 'white', **axis_font)

# Update function that gets called in each step
def update(num, x, y, line):
    ball_number = int(x[num])
    innings = 1 if ball_number < 121 else 2
    line.set_data(x[:num], y[:num]) 
    t1.set_text("\nSL Win Probability: " + '{:.2f}'.format(round(y[num], 2)) + "%\n")
    t2.set_text("\nInnings: " + str(innings) + "| Bat: " + get_batting_team(ball_number) + " | Score: " + get_text_styled_score(runs[num], wickets[num])
                + " | Overs: " + get_over_number(ball_number) + " | Run Rate: " + '{:.2f}'.format(run_rate[num]) + get_to_win_str(ball_number, to_win[num]) + "\n")
    t3.set_text("\nAUS Win Probability: " + '{:.2f}'.format(round(100-y[num], 2)) + "%\n")
    result.set_text(str(outcomes[num]))
    result.set_backgroundcolor(get_color(outcomes[num]))
    result.set_position(get_outcome_position(ball_number))
    return line,t1,t2,t3,
  
# Helper Functions
def get_batting_team(ball):
    if ball < 121:
        return "AUS"
    else:
        return "SL"

def get_over_number(ball):
    if ball < 121:
        over_str = f'{str(ball//6):>2}' + "." + str(ball%6) + " (Balls: " + f'{str(ball):>3}' + ")"
        return over_str
    else:
        b = ball - 120
        over_str = f'{str(b//6):>2}' + "." + str(b%6) + " (Balls: " + f'{str(ball):>3}' + ")"
        return over_str
    
def get_color(outcome):
    if outcome == '0':
        return '#433E47'
    elif outcome == '1':
        return '#35DF91'
    elif outcome == '2':
        return '#FAC762'
    elif outcome == '3':
        return '#FAC762'
    elif outcome == '4':
        return '#009944'
    elif outcome == '5':
        return '#009944'
    elif outcome == '6':
        return '#7A41D8'
    else:
        return '#E44242'

def get_outcome_position(ball):
    if ball < 121:
        return (0.25, 0.95)
    else:
        return (0.75, 0.95)
    
def get_text_styled_score(score, wickets):
    return str(score).rjust(3) + " / " + str(wickets)

def get_to_win_str(ball, to_win):
    if ball > 120:
        return " | To Win: " + str(int(to_win)).rjust(3)
    else:
        return ""

In [7]:
ani = animation.FuncAnimation(fig, update, frames = len(x), fargs=[x, y, line],
                              interval=200, blit=True)

In [10]:
# Show the animated plot

plt.show()

In [9]:
# Save the animation
metadata = dict(title="Sri Lanka's T20I win Probability Variation vs Australia (11th June 2022)", artist="Author's Name",
                comment=':)')
dpi = 300
writer = animation.writers['ffmpeg'](fps=10, bitrate = 10000)
Path('output').mkdir(exist_ok=True)
ani.save('output/plot.mp4', writer=writer, dpi=dpi)

In [11]:
# A basic animated plot without any styling

# x = ball
# y = prob

# fig, ax = plt.subplots()
# line, = ax.plot(x, y, color='k')


# def update(num, x, y, line):
#     plt.xticks(np.arange(12, 250, 12))
#     line.set_data(x[:num], y[:num])
#     line.axes.axis([0, 250, 0, 100])
#     return line,

# ani = animation.FuncAnimation(fig, update, len(x), fargs=[x, y, line],
#                               interval=25, blit=False)
# # ani.save('test.mp4')
# plt.show()

In [12]:
# A nice little code snippet for getting the available list of font families in the system

# import matplotlib.font_manager
# from IPython.core.display import HTML

# def make_html(fontname):
#     return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

# code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

# HTML("<div style='column-count: 2;'>{}</div>".format(code))